In [2]:
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "09_deploying/09b_rest.ipynb"
_nb_title = "Predictions using a REST endpoint"

### no need to change any of this
_nb_safeloc = _nb_loc.replace('/', '%2F')
md("""
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name={1}&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F{2}&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F{2}">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/{0}">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/{0}">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>
""".format(_nb_loc, _nb_title, _nb_safeloc))


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=Predictions using a REST endpoint&url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fblob%2Fmaster%2F09_deploying%2F09b_rest.ipynb&download_url=https%3A%2F%2Fgithub.com%2FGoogleCloudPlatform%2Fpractical-ml-vision-book%2Fraw%2Fmaster%2F09_deploying%2F09b_rest.ipynb">
    <img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/> Run in AI Platform Notebook</a>
  </td>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/GoogleCloudPlatform/practical-ml-vision-book/blob/master/09_deploying/09b_rest.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/GoogleCloudPlatform/practical-ml-vision-book/blob/master/09_deploying/09b_rest.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/09_deploying/09b_rest.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>


# Predictions using a REST endpoint

In this notebook, we start from an already trained and saved model (as in Chapter 7).
For convenience, we have put this model in a public bucket in gs://practical-ml-vision-book/flowers_5_trained

We deploy this model to a REST endpoint, and then show how to invoke the model using POST operations.

## REST endpoint

In [3]:
!cat ./caip_deploy.sh

#!/bin/bash

MODEL_NAME=flowers
VERSION_NAME=rest

REGION='us-central1'  # make sure you have GPU/TPU quota in this region
BUCKET='ai-analytics-solutions-kfpdemo' # for staging
MODEL_LOCATION="gs://practical-ml-vision-book/flowers_5_trained"

if [[ $(gcloud ai-platform models list --format='value(name)' | grep $MODEL_NAME) ]]; then
    echo "The model named $MODEL_NAME already exists."
else
    # create model
    echo "Creating $MODEL_NAME model now."
    gcloud ai-platform models create --regions=$REGION $MODEL_NAME
fi

if [[ $(gcloud ai-platform versions list --model $MODEL_NAME --format='value(name)' | grep $VERSION_NAME) ]]; then
    echo "Deleting already the existing model $MODEL_NAME:$VERSION_NAME ... "
    gcloud ai-platform versions delete --model=$MODEL_NAME $VERSION_NAME
    echo "Please run this script again if you don't see a Creating message ... "
    sleep 2
fi

# create model
echo "Creating $MODEL_NAME:$VERSION_NAME"
gcloud ai-platform versions create --model=$MODEL_NAM

In [44]:
!./caip_deploy.sh

Using endpoint [https://ml.googleapis.com/]
The model named flowers already exists.
Using endpoint [https://ml.googleapis.com/]
Creating flowers:rest
Using endpoint [https://ml.googleapis.com/]


To take a quick anonymous survey, run:
  $ gcloud survey



## JSON request

In [4]:
%%writefile request.json
{
    "instances": [
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}

Overwriting request.json


In [5]:
!gcloud ai-platform predict --model=flowers --version=rest --json-request=request.json

Using endpoint [https://ml.googleapis.com/]
FLOWER_TYPE_INT  FLOWER_TYPE_STR  PROBABILITY
1                dandelion        0.619152
1                dandelion        0.999984
0                daisy            0.995083
4                tulips           0.975186
4                tulips           0.954917


## sending over HTTP Post

In [6]:
## invoke from Python
from oauth2client.client import GoogleCredentials
import requests
import json

PROJECT = 'ai-analytics-solutions' # CHANGE
MODEL_NAME = 'flowers'
MODEL_VERSION = 'rest'

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = 'https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict' \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {'Authorization': 'Bearer ' + token }
data = {
    "instances": [
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{"predictions": [{"flower_type_str": "dandelion", "flower_type_int": 1, "probability": 0.6191519498825073}, {"flower_type_str": "dandelion", "flower_type_int": 1, "probability": 0.9999843835830688}, {"flower_type_str": "daisy", "flower_type_int": 0, "probability": 0.9950828552246094}, {"flower_type_str": "tulips", "flower_type_int": 4, "probability": 0.9751859307289124}, {"flower_type_str": "tulips", "flower_type_int": 4, "probability": 0.954916775226593}]}'


## CAIP Batch prediction

In [7]:
%%writefile batchinputs.json
{"filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg"}
{"filenames": "gs://cloud-ml-data/img/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"}
{"filenames": "gs://cloud-ml-data/img/flower_photos/daisy/9299302012_958c70564c_n.jpg"}
{"filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"}
{"filenames": "gs://cloud-ml-data/img/flower_photos/tulips/8713397358_0505cc0176_n.jpg"}

Overwriting batchinputs.json


In [ ]:
%%bash
BUCKET='ai-analytics-solutions-kfpdemo' # CHANGE
REGION="us-central1"
INPUT=gs://${BUCKET}/tmp/batchinputs.json
OUTPUT=gs://${BUCKET}/tmp/batchoutputs.json
gsutil cp batchinputs.json $INPUT
gsutil -m rm -rf $OUTPUT 
gcloud ai-platform jobs submit prediction flowers_batchpred_$(date -u +%y%m%d_%H%M%S) \
  --data-format=TEXT --region ${REGION} \
  --input-paths=$INPUT \
  --output-path=$OUTPUT \
  --model=flowers --version=rest

## Invoking from Apache Beam

In [21]:
import apache_beam as beam
from oauth2client.client import GoogleCredentials
import requests
import json

class ModelPredict:
    def __init__(self, project, model_name, model_version):
        self._api = ('https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict'
         .format(project, model_name, model_version))        
        
    def __call__(self, filenames):        
        token = GoogleCredentials.get_application_default().get_access_token().access_token
        if isinstance(filenames, str):
            # only one element, put it into a batch of 1
            data = {
                "instances": [
                    {"filenames": filenames}
                ]
            }
        else:
            data = {
                "instances": []
            }
            for f in filenames:
                data['instances'].append({
                    "filenames" : f
                })
        # print(data)
        headers = {'Authorization': 'Bearer ' + token }
        response = requests.post(self._api, json=data, headers=headers)
        response = json.loads(response.content.decode('utf-8'))
        # print(response)
        if isinstance(filenames, str):
            result = response['predictions'][0]
            result['filename'] = filenames
            yield result
        else:
            for (a,b) in zip(filenames, response['predictions']):
                result = b
                result['filename'] = a
                yield result


PROJECT = 'ai-analytics-solutions' # CHANGE
MODEL_NAME = 'flowers'
MODEL_VERSION = 'rest'

with beam.Pipeline() as p:    
    (p 
     | 'input' >> beam.Create([
        'gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg',
        'gs://cloud-ml-data/img/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg',
        'gs://cloud-ml-data/img/flower_photos/daisy/9299302012_958c70564c_n.jpg',
        'gs://cloud-ml-data/img/flower_photos/tulips/8733586143_3139db6e9e_n.jpg',
        'gs://cloud-ml-data/img/flower_photos/tulips/8713397358_0505cc0176_n.jpg'
     ]) 
     | 'batch' >> beam.BatchElements(min_batch_size=2, max_batch_size=3)
     | 'addpred' >> beam.FlatMap( ModelPredict(PROJECT, MODEL_NAME, MODEL_VERSION) )
     | 'write' >> beam.Map(print)
    )

{'flower_type_int': 1, 'flower_type_str': 'dandelion', 'probability': 0.6191519498825073, 'filename': 'gs://cloud-ml-data/img/flower_photos/dandelion/9818247_e2eac18894.jpg'}
{'flower_type_int': 1, 'flower_type_str': 'dandelion', 'probability': 0.9999843835830688, 'filename': 'gs://cloud-ml-data/img/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg'}
{'flower_type_int': 0, 'flower_type_str': 'daisy', 'probability': 0.9950828552246094, 'filename': 'gs://cloud-ml-data/img/flower_photos/daisy/9299302012_958c70564c_n.jpg'}
{'flower_type_int': 4, 'flower_type_str': 'tulips', 'probability': 0.9751859307289124, 'filename': 'gs://cloud-ml-data/img/flower_photos/tulips/8733586143_3139db6e9e_n.jpg'}
{'flower_type_int': 4, 'flower_type_str': 'tulips', 'probability': 0.954916775226593, 'filename': 'gs://cloud-ml-data/img/flower_photos/tulips/8713397358_0505cc0176_n.jpg'}


## License
Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.